# Laboratorio Optimización No. 4

> Daniel Alvarez - 201911320  
Maria Vieda - 201923359

- Importación de librerias

In [1]:
import numpy as np
from scipy.linalg import solve
from itertools import combinations
import pandas as pd
from IPython.display import display
from scipy.optimize import linprog, minimize
import time

- Funciones Recursivas

> Se realiza una función de truncamiento para encontrar cual es el error de aproximación entre la razon de redondeo de las soluciones,  
ya que, algunas de estas componen valores similares pero no iguales.

In [13]:
def truncate(n, decimals=0):
    # have the specific quantity of the decimals
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

> Se plantea el desarrollo de una funcion de limpieza para el diccionario que elimine las posibles soluciones repetidas del diccionario,  
sin embargo, para la función usada solver, no acota las soluciones a un valor proximo por lo que se veran soluciones donde se tiende a cero  
derecha y por izquierda como no iguales.

In [14]:
def remover_duplicados(objetos):
    resultado = {}
    for k, v in objetos.items():
        if v not in resultado.values():
            resultado[k] = v
    return resultado

> Se crea una función de paso que me entregue el vector de la fila n, por lo tanto, al tener 1 en su diagonal es equivalente a tener  
un vector de la forma:
$$x = [x_{1},x_{2},...,x_{n}]$$
>Donde
$$x_{i} = 0\;\; i\neq n$$ 

In [35]:
def vector_base(d,n):
    # v2 vector of zeros to fill  
    v2 = np.zeros(d)
    # v1 vector of ones to get the identity vector
    v1 = np.ones(d)
    for i in range(len(v2)):
        if i == n:
            v2[i] = v1[i]
    return v2 

- Punto 1

> Para el desarrollo del primer punto se nos solicita construir una función que reciba como parametros una matriz A, con n variables libres  
con la restricción:
$$n>m$$
>Donde existen variables $s_{1}$, $s_{2}$, ..., $s_{n}$ que se comportan como variables de holguras o variables libres para este sistema, se plantea como un algoritmo que encuentre todas las soluciones basicas sean óptimas o no, sin importar los valores de $x_{i}$, para esto se extraen las posibles combinaciones de que m-n variables sean cero, actuando como un sistema cerrado, por lo que, se acotan las soluciones de las matrices con:
$$det(A)\neq 0$$
> Pudiendo obtener las soluciones del sistema:
$$Ax=b$$
>Donde A es una matriz cuadrada en cada iteración de la combinatoria.  
> Para encontrar la primera matriz tomando como ejemplo la iteración $x_{1}$=$x_{2}$=0, para el sistema, pudiendo tener una matriz de 3x3,  
donde la solución se puede encontrar de diferentes formas, aparte para encontrar las bases se realizo la misma operación expresada como:
$$Ib_{2}=x$$
> Donde I es la matriz identidad y x es nuestro vector solución del sistema Ax =b.

In [161]:
def sol_basicas(A,b):
    # get columns and rows
    f, c = A.shape
    fv = c - f
    print(A)
    print(b)
    # get all the combinations of the matrix with c-f fv (free variables)
    comb = combinations(range(c), c-f)
    # create the dictionary
    dire = {'Solucion':[], 'Base':[]}
    # create the combinations of the basic variables
    c_s = set(np.array(range(c)))
    # get the matrix of A
    # build the base
    for i in list(comb):
        no_b = set(i)
        c_m = c_s^no_b
        # we obtain the matrix of the combination iteration
        B = A[:, list(c_m)]
        # create a vector of zeros with c positions
        sol = np.zeros(c)
        # same
        base = sol
        # same
        no_base = sol
        # we get the identity to found the bases of the solution
        m = np.identity(c-fv)
        # we get the identity to get the real bases in terms of c positions
        m2 = np.identity(c)
        # we get an identify of cxc to multplicate 
        iden = np.identity(c)
        # we condition that the matrix B have solution
        if np.linalg.det(B) != 0:
            # use solve to the system Bx=b
            x = solve(B, b)
            # use solve to the system Ico=x to get the bases
            co = solve(m,x)
            # x = x.astype(int)
            for j in range(c-fv):
                valor = list(c_m)[j]
                sol[valor] = round(x[j],3)
                base[valor] = round(co[j],3)
                for k in range(c):
                    if k != valor:
                        m2[:,k] = no_base
                #  print(m2)
            dire['Solucion'].append(sol)
            dire['Base'].append(m2*iden)
    return dire

> Definimos las matrices $A_{1}$ y $A_{2}$, junto con el vector b que se aplicaran a la funcion para encontrar las soluciones basicas del problema de PL.

In [128]:
# vectors of prove
# A is a matrix
A1 = np.array([[1, 1, 1, 1, 1],
             [1, 2, 3, 4, 5],
             [4, 3, 3, 3, 4]])
# A2 is a matrix of /
A2 = np.array([[1, 1, 1, 1, 1],
               [1, 2, 3, 4, 5],
               [2, 3, 4, 5, 6]])
# b is the vector result
b = np.array([1, 2, 3])

> Realizamos la prueba para la matriz $A_{1}$.

In [131]:
prueba = sol_basicas(A1,b)
print(prueba)

{'Solucion': [array([ 0.,  0.,  2., -1., -0.]), array([ 0.,  1.,  0.,  0., -0.]), array([ 0.,  1.,  0.,  0., -0.]), array([ 0.5,  0. ,  0. ,  1. , -0.5]), array([ 0.25,  0.  ,  1.  ,  0.  , -0.25]), array([ 0.,  0.,  2., -1.,  0.]), array([ 0.,  1.,  0.,  0., -0.]), array([ 0.,  1.,  0., -0.,  0.]), array([ 0.,  1., -0.,  0.,  0.])], 'Base': [array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  2.,  0.,  0.],
       [-0., -0., -0., -1., -0.],
       [-0., -0., -0., -0.,  0.]]), array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0., -0.,  0.]]), array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0., -0.,  0.]]), array([[ 0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0

> Realizamos la prueba para la matriz $A_{2}$  
Nota: Tomemos en cuenta que para este sistema, al ser una matriz de rango deficiente 
solver no representa la función más correcta para hacer la aproximación de la solución por lo tanto obtenemos una advertencia.

In [129]:
prueba2 = sol_basicas(A2,b)
print(prueba2)

{'Solucion': [array([ 0.,  1.,  0.,  0., -0.]), array([ 0.,  1.,  0.,  0., -0.])], 'Base': [array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0., -0.,  0.]]), array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0., -0.,  0.]])]}


C:\Users\danie\AppData\Local\Temp\ipykernel_10228\1600866937.py:26: LinAlgWarning: Ill-conditioned matrix (rcond=1.85037e-17): result may not be accurate.
  x = solve(B, b)
C:\Users\danie\AppData\Local\Temp\ipykernel_10228\1600866937.py:26: LinAlgWarning: Ill-conditioned matrix (rcond=9.25186e-18): result may not be accurate.
  x = solve(B, b)


In [133]:
pd.set_option('max_colwidth',75)
sol = sol_basicas(A2,b)
df_sol = pd.DataFrame(sol)
print('\n Soluciones básicas (Matriz de rango completo): \n')
display(df_sol)


 Soluciones básicas (Matriz de rango completo): 



C:\Users\danie\AppData\Local\Temp\ipykernel_10228\4000819965.py:24: LinAlgWarning: Ill-conditioned matrix (rcond=1.85037e-17): result may not be accurate.
  x = solve(B, b)
C:\Users\danie\AppData\Local\Temp\ipykernel_10228\4000819965.py:24: LinAlgWarning: Ill-conditioned matrix (rcond=9.25186e-18): result may not be accurate.
  x = solve(B, b)


,Solucion,Base
0,"[0.0, 1.0, 0.0, 0.0, -0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, ..."
1,"[0.0, 1.0, 0.0, 0.0, -0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, ..."


In [134]:
pd.set_option('max_colwidth',75)
sol = sol_basicas(A1,b)
df_sol = pd.DataFrame(sol)
print('\n Soluciones básicas (Matriz de rango completo): \n')
display(df_sol)


 Soluciones básicas (Matriz de rango completo): 



,Solucion,Base
0,"[0.0, 0.0, 2.0, -1.0, -0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, ..."
1,"[0.0, 1.0, 0.0, 0.0, -0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, ..."
2,"[0.0, 1.0, 0.0, 0.0, -0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, ..."
3,"[0.5, 0.0, 0.0, 1.0, -0.5]","[[0.5, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, ..."
4,"[0.25, 0.0, 1.0, 0.0, -0.25]","[[0.25, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0,..."
5,"[0.0, 0.0, 2.0, -1.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, ..."
6,"[0.0, 1.0, 0.0, 0.0, -0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, ..."
7,"[0.0, 1.0, 0.0, -0.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, ..."
8,"[0.0, 1.0, -0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [-0.0, -0.0, 0.0..."


> La diferencia radica en que al tener el rango completo en 9 de sus 10 matrices combinadas se encuentran soluciones a diferencia de la matriz 2 solo se encuentran 2 de las 10. Eliminando las que son iguales tambien podemos notar que para el sistema de la matriz 2 existe una esquina la cual es una solución basica o esquina de la geometria asociada, a diferencia de la 1 que tiene 4 vertices, 4 soluciones basicas. 

- Punto 2

> El problema consiste en 3 plantas de producción de energía eléctrica por medio de la quema de combustibles fósiles como lo son el carbón, el petróleo y el gas natural. Por cada uno de los combustibles se tienen la siguiente información, 1 tonelada de carbón, genera 600 kilovatios/hora, emite 20 unidades de bióxido de azufre, 15 unidades de partículas suspendidas y su costo es de 200 dólares; 1 tonelada de petróleo, genera 550 kilovatios hora, emite 18 unidades de bióxido de azufre, 12 unidades de partículas suspendidas y su costo es de 220 dólares; 1 tonelada de gas, genera 500 kilovatios/hora, emite 15 unidades de bióxido de azufre, 10 unidades de partículas suspendidas y  cuesta 250 dólares. Se conocen ahora las restricciones que impuso la oficina de protección ambiental, las cuales limitan las emisiones de bióxido de azufre a no más de 60 unidades y las partículas suspendidas a no más de 75 diarias. Por último, se cuenta con 2000 USD para el pago total de la realización del proyecto. ¿Cuánto combustible de cada clase debe comprar para maximizar la cantidad de energía generada?

> Definimos las variables de costo:

$$x_{1}\rightarrow\;\; Cantidad\;de\;Carbón$$ 
$$x_{2}\rightarrow\;\; Cantidad\;de\;Petróleo$$ 
$$x_{3}\rightarrow\;\; Cantidad\;de\;Gas$$ 

> Del problema se asume que se busca maximizar la cantidad de energía eléctrica generada con respecto al combustible que consume cada planta (carbón, petróleo y gas) por lo que se sabe que el resultado esperado de dicho proceso estará sujeto a múltiples restricciones, las cuales delimitan la relación entre el nivel de producción (kilovatios/hora) y el costo económico (USD) que implica la generación los mismo. 
Por lo que, se propone la siguiente función de costos:
$$P(x_{1}, x_{2}, x_{3}) = 600x_{1} + 550x_{2} + 500x_{3}$$

> Se plantea el ejercicio de PL con las siguientes restricciones

- S.A
$$20x_{1} + 18x_{2} + 15x_{3} \leq 60$$
$$15x_{1} + 12x_{2} + 10x_{3} \leq 75 $$
$$200x_{1} + 220x_{2} + 250x_{3} \leq 2000 $$
> La última restricción es que ninguna producción puede ser negativa
$$x_{n} \geq 0 \therefore n \in [1,2,3]$$

> De las tres ecuaciones de restricciones que entienden como, condiciones que se deben cumplir de forma que se obtenga el resultado esperado sobre la cantidad de combustible para maximizar el proceso de producción energética. Estas restricciones están ligadas a la oficina de protección ambiental la cual restringe las emisiones diarias tanto de partículas suspendidas como bióxido de azufre, las cuales no pueden superar los valores de 75 y 60 la cual representarían las dos primeras ecuaciones. La tercera ecuación cuenta con la restricción del presupuesto para la realización del proyecto el cual no debe superar los 2000 dólares. 

> Se plantea el problema de forma estandar:
$$min \;\; -600x_{1} -550x_{2} - 500x_{3}$$
- S.A
$$20x_{1} + 18x_{2} + 15x_{3} + s_{1} = 60$$
$$15x_{1} + 12x_{2} + 10x_{3} + s_{2} = 75 $$
$$200x_{1} + 220x_{2} + 250x_{3} + s_{3}  = 2000 $$
> La última restricción es que ninguna producción puede ser negativa
$$x_{n} \geq 0 \therefore n \in [1,2,3]$$
$$s_{n} \geq 0 \therefore n \in [1,2,3]$$


> Se plantea el problema de la misma forma sin las variables de holgura.
$$max \;\; 600x_{1} + 550x_{2} + 500x_{3}$$
- S.A
$$20x_{1} + 18x_{2} + 15x_{3}  \leq 60$$
$$15x_{1} + 12x_{2} + 10x_{3} \leq 75 $$
$$200x_{1} + 220x_{2} + 250x_{3} \leq 2000 $$
> La última restricción es que ninguna producción puede ser negativa
$$x_{n} \geq 0 \therefore n \in [1,2,3]$$

> Se nos solicita implementar una función que reciba como parámetros las restricciones mencionadas anteriormente de forma que encuentre todas las soluciones básicas; y así sea posible verificar la factibilidad de todas estas soluciones, es decir, que tengan un valor mayor a 0.

In [2]:
c_ex = np.array([600,550,500]) # x_1 x_2 x_3
A3 = np.array([[20, 18, 15, 1, 0, 0],
              [15, 12, 10, 0, 1, 0],
              [200, 220, 250, 0, 0, 1]])
b3 = np.array([60, 75, 2000])

In [161]:
def sol_basicas2(A,b,P):
    # get columns and rows
    f3, c3 = A.shape
    fv3 = c3 - f3
    # get all the combinations of the matrix with c-f fv (free variables)
    comb = combinations(range(c3), c3-f3)
    # create the dictionary
    dire = {'Solucion':[], 'Base':[], 'Tipo':[], 'valor':[]}
    # create the combinations of the basic variables
    c_s = set(np.array(range(c3)))
    # get the matrix of A
    filtro = np.array
    # build the base
    tipo1 = 'factible'
    tipo2 = 'no-factible'
    tipo = ' '
    for i in list(comb):
        no_b = set(i)
        c_m = c_s^no_b
        # we obtain the matrix of the combination iteration
        B = A[:, list(c_m)]
        # create a vector of zeros with c positions
        sol = np.zeros(c3)
        # same
        base = sol
        # same
        no_base = sol
        # set a flag to change if the condition xn>0 is not possible
        bandera = True
        tipo = tipo1
        # we get the identity to found the bases of the solution
        m = np.identity(c3-fv3)
        # we get the identity to get the real bases in terms of c positions
        m2 = np.identity(c3)
        # we get an identify of cxc to multplicate 
        iden = np.identity(c3)
        # we condition that the matrix B have solution
        if np.linalg.det(B) != 0:
            # use solve to the system Bx=b
            x = solve(B, b)
            # use solve to the system Ico=x to get the bases
            co = solve(m,x)
            # x = x.astype(int)
            for j in range(c3-fv3):
                if x[j] < 0: 
                    tipo = tipo2
                valor = list(c_m)[j]
                sol[valor] = round(x[j],3)
                base[valor] = round(co[j],3)
                for k in range(c3):
                    if k != valor:
                        m2[:,k] = no_base
                    #  print(m2)
            dire['Solucion'].append(sol)
            dire['Base'].append(m2*iden)
            dire['Tipo'].append(tipo)
            t = np.zeros(c3-fv3)
            for u in range(len(P)-1):
                t[u] = P[u]*sol[u+1]-100
            dire['valor'].append(sum(t))
    return dire

In [162]:
# vectores de prueba 

start = time.time()
prueba3 =  sol_basicas2(A3,b3,c_ex)
end = time.time()
print(end-start)
print(prueba3)

0.0059986114501953125
{'Solucion': [array([   0.,    0.,    0.,   60.,   75., 2000.]), array([   0.,    0.,    4.,    0.,   35., 1000.]), array([  0. ,   0. ,   7.5, -52.5,   0. , 125. ]), array([  0.,   0.,   8., -60.,  -5.,   0.]), array([   0.   ,    3.333,    0.   ,    0.   ,   35.   , 1266.667]), array([  0.  ,   6.25,   0.  , -52.5 ,   0.  , 625.  ]), array([   0.   ,    9.091,    0.   , -103.636,  -34.091,    0.   ]), array([ 0.00000000e+00, -6.30503948e+16,  7.56604737e+16,  0.00000000e+00,
        0.00000000e+00, -5.04403158e+18]), array([  0. , -12.5,  19. ,   0. ,  35. ,   0. ]), array([  0.   ,  -1.563,   9.375, -52.5  ,   0.   ,   0.   ]), array([   3.,    0.,    0.,    0.,   30., 1400.]), array([   5.,    0.,    0.,  -40.,    0., 1000.]), array([  10.,    0.,    0., -140.,  -75.,    0.]), array([  21.,    0.,  -24.,    0.,    0., 3800.]), array([-7.5,  0. , 14. ,  0. , 47.5,  0. ]), array([ -0.714,   0.   ,   8.571, -54.286,   0.   ,   0.   ]), array([  21.,  -20.,    0.,

C:\Users\danie\AppData\Local\Temp\ipykernel_11340\3491470340.py:40: LinAlgWarning: Ill-conditioned matrix (rcond=2.4557e-20): result may not be accurate.
  x = solve(B, b)


In [163]:
pd.set_option('max_colwidth',75)
prueba3 = sol_basicas2(A3,b3,c_ex)
df_sol = pd.DataFrame(prueba3)
print('\n Soluciones básicas (Matriz de rango completo): \n')
display(df_sol)


 Soluciones básicas (Matriz de rango completo): 



C:\Users\danie\AppData\Local\Temp\ipykernel_11340\3491470340.py:40: LinAlgWarning: Ill-conditioned matrix (rcond=2.4557e-20): result may not be accurate.
  x = solve(B, b)


,Solucion,Base,Tipo,valor
0,"[0.0, 0.0, 0.0, 60.0, 75.0, 2000.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, ...",factible,-2.000000e+02
1,"[0.0, 0.0, 4.0, 0.0, 35.0, 1000.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, ...",factible,2.000000e+03
2,"[0.0, 0.0, 7.5, -52.5, 0.0, 125.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, ...",no-factible,3.925000e+03
3,"[0.0, 0.0, 8.0, -60.0, -5.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, ...",no-factible,4.200000e+03
4,"[0.0, 3.333, 0.0, 0.0, 35.0, 1266.667]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 3.333, 0.0, 0.0, 0.0, 0.0], [0.0...",factible,1.799800e+03
5,"[0.0, 6.25, 0.0, -52.5, 0.0, 625.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 6.25, 0.0, 0.0, 0.0, 0.0], [0.0,...",no-factible,3.550000e+03
6,"[0.0, 9.091, 0.0, -103.636, -34.091, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 9.091, 0.0, 0.0, 0.0, 0.0], [0.0...",no-factible,5.254600e+03
7,"[0.0, -6.305039478318694e+16, 7.566047373982434e+16, 0.0, 0.0, -5.04403...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.0, -6.305039478318694e+16, -0.0, -...",no-factible,3.783024e+18
8,"[0.0, -12.5, 19.0, 0.0, 35.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.0, -12.5, -0.0, -0.0, -0.0, -0.0],...",no-factible,2.750000e+03
9,"[0.0, -1.563, 9.375, -52.5, 0.0, 0.0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.0, -1.563, -0.0, -0.0, -0.0, -0.0]...",no-factible,4.018450e+03


> Como podemos observar la solucion factible optima se encuentra en la segunda iteración, con una función de costo P($x_{1}$,$x_{2}$,$x_{3}$)= 2000, comparandolo con los algoritmos 'revised_simplex', 'interior_point' y 'simplex'

In [137]:
# P -> función de costo
# coeficientes de P 
# los coeficientes teniendo en cuenta que todos los x son el tipo de recurso 
c_ex = np.array([600,550,500]) # x_1 x_2 x_3

# constantes de inecuaciones x_1 x_2 x_3 = 3 variables de control
# representan 3 restricciones
# [1-3] representan la cantidad máxima de producción por planta
# [4-6] representan la cantidad máxima de pies cuadrados que ocupan por planta
# [7-9] representan la cantidad máxima estimada de venta por tipo de producto
A = np.array([[20, 18, 15],
              [15, 12, 10],
              [200, 220, 250]])

# se cre el vector solucion de la matriz
b = np.array([60, 75, 2000])
# restricciones de variables no negativas, (no puede existir producción negativa)
bounds_ex = [(  0,    None)]*3
# Solución de problema a traves de simplex
# se toma el vector C como negativo para encontrar el minimo de la funcion invertida, osea el máximo
start = time.time()
res_ex2 = linprog(-c_ex, A_ub=A, b_ub=b, bounds=bounds_ex, method='revised simplex')
end = time.time()
# se toma las soluciones probables del sistema 
print(end-start)
res_ex2

0.003997802734375


     con: array([], dtype=float64)
     fun: -2000.0
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([   0.,   35., 1000.])
  status: 0
 success: True
       x: array([0., 0., 4.])

In [139]:
# P -> función de costo
# coeficientes de P 
# los coeficientes teniendo en cuenta que todos los x son el tipo de recurso 
c_ex = np.array([600,550,500]) # x_1 x_2 x_3

# constantes de inecuaciones x_1 x_2 x_3 = 3 variables de control
# representan 3 restricciones
# [1-3] representan la cantidad máxima de producción por planta
# [4-6] representan la cantidad máxima de pies cuadrados que ocupan por planta
# [7-9] representan la cantidad máxima estimada de venta por tipo de producto
A = np.array([[20, 18, 15],
              [15, 12, 10],
              [200, 220, 250]])

# se cre el vector solucion de la matriz
b = np.array([60, 75, 2000])
# restricciones de variables no negativas, (no puede existir producción negativa)
bounds_ex = [(  0,    None)]*3
# Solución de problema a traves de simplex
# se toma el vector C como negativo para encontrar el minimo de la funcion invertida, osea el máximo
start = time.time()
res_ex2 = linprog(-c_ex, A_ub=A, b_ub=b, bounds=bounds_ex, method='interior-point')
end = time.time()
# se toma las soluciones probables del sistema 
print(end-start)
res_ex2

0.015993118286132812


     con: array([], dtype=float64)
     fun: -1999.999999995471
 message: 'Optimization terminated successfully.'
     nit: 8
   slack: array([7.4685147e-11, 3.5000000e+01, 1.0000000e+03])
  status: 0
 success: True
       x: array([2.99286723e-11, 8.88531678e-13, 4.00000000e+00])

In [140]:
# P -> función de costo
# coeficientes de P 
# los coeficientes teniendo en cuenta que todos los x son el tipo de recurso 
c_ex = np.array([600,550,500]) # x_1 x_2 x_3

# constantes de inecuaciones x_1 x_2 x_3 = 3 variables de control
# representan 3 restricciones
# [1-3] representan la cantidad máxima de producción por planta
# [4-6] representan la cantidad máxima de pies cuadrados que ocupan por planta
# [7-9] representan la cantidad máxima estimada de venta por tipo de producto
A = np.array([[20, 18, 15],
              [15, 12, 10],
              [200, 220, 250]])

# se cre el vector solucion de la matriz
b = np.array([60, 75, 2000])
# restricciones de variables no negativas, (no puede existir producción negativa)
bounds_ex = [(  0,    None)]*3
# Solución de problema a traves de simplex
# se toma el vector C como negativo para encontrar el minimo de la funcion invertida, osea el máximo
start = time.time()
res_ex2 = linprog(-c_ex, A_ub=A, b_ub=b, bounds=bounds_ex, method='simplex')
end = time.time()
# se toma las soluciones probables del sistema 
print(end-start)
res_ex2

0.006997346878051758


     con: array([], dtype=float64)
     fun: -2000.0
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([   0.,   35., 1000.])
  status: 0
 success: True
       x: array([0., 0., 4.])

> Señor gerente, las soluciones principales que se rechazan o son clasificadas como “no factibles” quieren decir que tienen un valor menor o igual 0, lo que indica que no se estaría llegando al límite máximo de esa planta en cuanto a producción. En este caso se busca obtener la planta que produzca mayor cantidad de energía eléctrica sin sobrepasar las restricciones dadas; por lo que se obtiene la solución con valor 004 correspondiente a la planta de gas natural, la cual entre todos los resultados es la solución que maximiza todo el proceso de la producción esto quiere decir, que esta dentro de los limites del presupuesto que usted nos indicó y además, cuenta con todas las normas impuestas por la oficina de protección ambiental sobre las emisiones de efecto invernadero. 

- Conclusión

> Después de revisar a través de múltiples algoritmos el tiempo de ejecución de los mismos evidenciamos como el valor que se obtiene en al implementar punto interior es 0.01599, con simplex obtuvimos 0.006997 y con simplex revisado se obtuvo 0.00399. Mientras que, con la solución propuesta por nosotros el tiempo que tardo este fue de 0.006998, por lo que al compararlo con los demás evidenciamos que el algoritmo nuestro es óptimo y eficiente.